# Computation Random Path

Objective:

- Compute Random path with MC simulation

More about Derivatives [here](https://drive.google.com/open?id=1Pz23oqSVdAnt2ruj5DhXOjgqLjFzy6Hf)

## Parameter of the model

- Heston parameters	6/29/2018
- Kappa:	0.5485872
- Theta:	0.0462418
- Sigma:	0.2252450
- Rho:	-0.5014077
- V0:	0.0190355
- OIS: 0.00144
- Dividend: 0.02
- Spot: 105.42
- current date: 6/29/2018
- valuation date: 1/19/2021

# Computation Random path


## Calcul Heston

**Calcul Heston**
$$
Z1 = N(0,1) \\
Zt = N(0,1) \\
Z3 = \rho * Z1 + (1-\sqrt{\rho})^{0.5} * Zt
$$

In [ ]:
import numpy as np

In [ ]:
seed = np.random.RandomState(123456)
rho = -0.5014077 
iteration = 5

In [ ]:
def Z(rho, iteration, seed):
    '''
    Calcule les parametres de correlation pour Heston
    '''
    Z1 = seed.normal(loc=0.0, scale=1.0, size = iteration)
    Zt = seed.normal(loc=0.0, scale=1.0, size = iteration)
    Z2 = rho * Z1 + np.power(1 - np.square(rho),0.5) * Zt
    return Z1, Z2

In [ ]:
Z(rho, iteration, seed)

## Random path

$$
Dst = Ds_t * (1 + (OIS - Dividend) *Dt )+ Dv_t * Dt ^{0.5} * Z2 \\
Dvt = Dv_t + \kappa * (\theta - Dv_t) * Dt + \sigma * Dv_t * Dt ^ {0.5} * Z1
$$

In [ ]:
def random_path(Z1, Z2, dS, dVt, ois, dividend, kappa, theta, sigma):
    '''
    Genere un array de la dimension du nombre d'iteration
    La diffusion peut etre calculee selon 3 facons
    - Normal
    '''
    dDt = 1/252
    #Z1, Z2 = Z()
    dS = dS * (1 + (ois - dividend) * dDt + np.power(dVt * dDt, 0.5) * Z2)
    dVt = dVt + kappa*(theta - dVt) * dDt + sigma * \
        np.power(dVt * dDt, 0.5) * Z1
    dVt = np.abs(dVt)
    return dS, dVt


In [ ]:
dS = 105.42
dVt = 0.0190355
ois = 0.00144
dividend =  0.02
kappa = 0.5485872
theta = 0.0462418
sigma = 0.2252450
Z1, Z2 = Z(rho, iteration, seed)

In [ ]:
random_path(
    Z1=Z1,
    Z2=Z2,
    dS=dS,
    dVt=dVt,
    ois=ois,
    dividend=dividend,
    kappa=kappa,
    theta=theta,
    sigma=sigma)

## Get maturity

Get number of days between two dates

In [ ]:
#!pip install workalendar

In [ ]:
from datetime import date
import pandas as pd
from workalendar.europe import France

In [ ]:
def date_transformed(current_date, valuation_date):
    """
            La fonction calcule le nombre de jours avant une date de
             constatation/maturité;
            la date maximum rescale (jour/delta) et la liste des dates rescales
             si elle existe
            Si la liste des valuations_date a une seule valeur, ie la date de
             maturité alors
            le code va dans except
    """
    dDt = 252
    
    try:
        days_to_T = [
            cal.get_working_days_delta(x, current_date) for x in valuation_date
        ]
        days_to_T_scaled = [
            cal.get_working_days_delta(x, current_date) / dDt for x in valuation_date
        ]
        T_scale = max(days_to_T_scaled)
        return days_to_T, T_scale, days_to_T_scaled
    except:
        days_to_T = cal.get_working_days_delta(valuation_date, current_date)
        T_scale = cal.get_working_days_delta(
            valuation_date, current_date) / dDt
        return days_to_T, T_scale


In [ ]:
current_date = "29/06/2018"
valuation_date = "19/01/2021"
cal = France()
current_date = pd.to_datetime(current_date, format = "%d/%m/%Y")
valuation_date = pd.to_datetime(valuation_date,
		 format = "%d/%m/%Y")

In [ ]:
date_transformed(current_date, valuation_date)

## Compute the Random Path with MC simulation

We want to store the random path and get the average of tge DS at maturity

In [ ]:
date_transformed_ = date_transformed(current_date, valuation_date)
days = np.array(date_transformed_[0])
maturity_days = np.max(days)
list_Ds = []
list_Dv = []

In [ ]:
days

Create an empty array with the first value of the model

In [ ]:
iteration = 10000
ds = np.asarray(np.repeat(dS,iteration))
dv = np.asarray(np.repeat(dVt,iteration))
path = [ds, dv]

Iterate

In [ ]:
for n in range(1, maturity_days + 1):
    Z1, Z2 = Z(rho, iteration, seed)
    path = random_path(Z1=Z1,
                       Z2=Z2,
                       dS= path[0],
                       dVt= path[1],
                       ois=ois,
                       dividend=dividend,
                       kappa=kappa,
                       theta=theta,
                       sigma=sigma
                      )
    
    list_Ds.append(path[0])
    list_Dv.append(path[1])
    
    ## Compute average last DS
    if n == maturity_days:
        avg = np.mean(path[0])

In [ ]:
avg

In [ ]:
list_Ds[0:][:1]

## Plot the Random path

In [ ]:
pd.DataFrame(np.transpose(list_Ds)).iloc[:10].transpose().plot()

## Compute DV

In [ ]:
pd.DataFrame(np.transpose(list_Dv)).iloc[:10].transpose().plot()

# Your turn 

## Random path with forward diffusion

$$
Dst = Ds_t * (1 + (forward_t - Dividend) *Dt )+ Dv_t * Dt ^{0.5} * Z2 \\
Dvt = Dv_t + \kappa * (\theta - Dv_t) * Dt + \sigma * Dv_t * Dt ^ {0.5} * Z1
$$

Here is the list of forward

| Dates     | Taux forwards |
| --------- | ------------- |
| 9/24/2018 | -0.36%        |
| 9/24/2019 | -0.33%        |
| 9/24/2020 | -0.14%        |
| 9/17/2021 | 0.07%         |

Hint:

Pass a list of date in `date_transformed` and check when `n` is in `days` and slice `forward_rate` accordingly.

Remember to change the fonction `random_path`

In [ ]:
list_value = ['24/9/2018', '24/9/2019', '24/9/2020', '17/9/2021']
valuation_date = pd.to_datetime(list_value,
		 format = "%d/%m/%Y")
date_transformed_ = date_transformed(current_date, valuation_date)
days = np.array(date_transformed_[0])
days

In [ ]:
forward_rate = [-0.0036, -0.0033, -0.0014, 0.007]
i = 0
for n in range(1, maturity_days + 1):
    ### Do something
    if n in days:
        i += 1
        print(forward_rate[i])